In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import time

In [22]:
headers_desktop = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
headers_mobile = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'}

In [23]:
# Read csv file
df = pd.read_csv('../EveryTraceOfFampayPeriod/famapp_unique_links.csv')
df

,Link
0,https://famapp.in/
1,https://famcard.me/
2,https://fampay.in/
3,https://fampay.in/about
4,https://fampay.in/blog
...,...
518,https://famapp.in/privacy
519,https://famapp.in/terms
520,https://famapp.in/contact
521,https://www.instagram.com/fam.india/


In [24]:
def get_metrics(url, headers):
    try:
        print(f'Checking site: {url}')
        start_time = time.time()

        # Send a request
        response = requests.get(url, headers=headers)

        elapsed_time = time.time() - start_time

        # Parse the HTML of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the title and description tags
        title_tag = soup.find('title')
        description_tag = soup.find('meta', {'name':'description'})

        # Find the viewport tag
        viewport_tag = soup.find('meta', {'name':'viewport'})

        print(f'Finished checking site: {url}, time: {elapsed_time}')

        # Return a dictionary with all metrics
        return {
            'url': url,
            'response_time': elapsed_time,
            'http_status': response.status_code,
            'title': title_tag.text if title_tag else None,
            'description': description_tag['content'] if description_tag else None, # type: ignore
            'is_mobile_friendly': viewport_tag is not None,
        }

    except Exception as e:
        print(f'Error checking site: {url}, error: {str(e)}')
        return None

In [25]:
# Define a function to check if a site is mobile friendly
def get_all_metrics(url):
    desktop_metrics = get_metrics(url, headers_desktop)
    mobile_metrics = get_metrics(url, headers_mobile)

    if desktop_metrics is None or mobile_metrics is None:
        return None

    # Combine the metrics from both versions
    combined_metrics = {
        'url': url,
        'desktop_response_time': desktop_metrics['response_time'],
        'mobile_response_time': mobile_metrics['response_time'],
        'desktop_http_status': desktop_metrics['http_status'],
        'mobile_http_status': mobile_metrics['http_status'],
        'desktop_title': desktop_metrics['title'],
        'mobile_title': mobile_metrics['title'],
        'desktop_description': desktop_metrics['description'],
        'mobile_description': mobile_metrics['description'],
        'is_mobile_friendly': desktop_metrics['is_mobile_friendly'] and mobile_metrics['is_mobile_friendly'],
    }

    print(f'Finished checking site: {url}')

    return combined_metrics

In [26]:
# Apply function to each url
df_metrics = pd.DataFrame([get_all_metrics(url) for url in df['Link'].dropna()])
df_metrics.to_csv('website_metrics.csv', index=False)

Checking site: https://famapp.in/
Finished checking site: https://famapp.in/, time: 0.22333002090454102
Checking site: https://famapp.in/
Finished checking site: https://famapp.in/, time: 0.12727069854736328
Finished checking site: https://famapp.in/
Checking site: https://famcard.me/
Finished checking site: https://famcard.me/, time: 0.2695047855377197
Checking site: https://famcard.me/
Finished checking site: https://famcard.me/, time: 0.2392745018005371
Finished checking site: https://famcard.me/
Checking site: https://fampay.in/
Finished checking site: https://fampay.in/, time: 0.2522594928741455
Checking site: https://fampay.in/
Finished checking site: https://fampay.in/, time: 0.24708819389343262
Finished checking site: https://fampay.in/
Checking site: https://fampay.in/about
Finished checking site: https://fampay.in/about, time: 0.2553226947784424
Checking site: https://fampay.in/about
Finished checking site: https://fampay.in/about, time: 0.2816956043243408
Finished checking s